## Homework 2: Naive Bayes

### CS 490A, UMass CICS, Fall 2022
### Due: September 28th at 11:59pm. 75 points total.

In this homework, you will implement and train a naive Bayes classifier on a collection of movie reviews in order to distinguish between positive and negative reviews.


##### How to submit this homework:
Write all the answers in this notebook. Once you are finished, you must submit the following to gradescope:
1. The generated PDF of your completed notebook
2. Your completed notebook `hw2.ipynb`

## Collaboration Declarations:
List here the name of any classmates that you had high-level discussions with about the homework.

**OPTIONAL: Your declarations here**

## Setup

In [1]:
import math
import os
from collections import Counter

We will use the `simple_tokenizer` function to tokenize our reviews.

In [2]:
def simple_tokenizer(text):
    """
    DO NOT MODIFY THIS FUNCTION
    
    Tokenizes text and returns its bag-of-words representation.
    A text is tokenized through lowercasing and splitting on whitespace.
    text - string representing the text to be tokenized
    
    Returns a Counter mapping each word to the number of times it appears in text
    """
    tokens = text.lower().split()
    bow = Counter(tokens)
    return bow

## Implementing Naive Bayes
The following code cell contains an *incomplete* implementation of a naive Bayes classifier. You will complete this implementation through the course of this assignment. Be sure to read the code and comments carefully.

In [3]:
class NaiveBayes:
    """A naive Bayes model for text classification."""

    def __init__(self, train_tsv, test_tsv, tokenizer):
        # Vocabulary is a set that stores every word seen in the training data
        self.vocab = set()
        self.tokenizer = tokenizer
        self.train_fn = train_tsv
        self.test_fn = test_tsv
        # class_total_doc_counts is a dictionary that maps a class (i.e., pos/neg) to
        # the number of documents in the trainning set of that class
        self.class_total_doc_counts = { "pos": 0.0,
                                        "neg": 0.0 }

        # class_total_word_counts is a dictionary that maps a class (i.e., pos/neg) to
        # the number of words in the training set in documents of that class
        self.class_total_word_counts = { "pos": 0.0,
                                         "neg": 0.0 }

        # class_word_counts is a dictionary of Counters. It maps a class (i.e.,
        # pos/neg) to a Counter of word counts.
        # For example:
        #    self.class_word_counts[POS_LABEL]['awesome']
        #    stores the number of times the word 'awesome' appears in documents
        #    of the positive class in the training documents.
        self.class_word_counts = { "pos": Counter(),
                                   "neg": Counter() }

    def train_model(self):
        """
        DO NOT MODIFY THIS FUNCTION
        
        This function processes the entire training set file.  It makes use of
        the tokenize_and_update_model function that you will implement.
        """
        
        with open(self.train_fn, encoding="utf-8") as reader:
            for line in reader:
                _, label, text = line.strip().split('\t')
                self.tokenize_and_update_model(text, label)
        self.report_statistics_after_training()

    
    def report_statistics_after_training(self):
        """
        DO NOT MODIFY THIS FUNCTION
        
        Report a number of statistics after training.
        """

        print ("REPORTING CORPUS STATISTICS")
        print ("NUMBER OF DOCUMENTS IN POSITIVE CLASS:", self.class_total_doc_counts["pos"])
        print ("NUMBER OF DOCUMENTS IN NEGATIVE CLASS:", self.class_total_doc_counts["neg"])
        print ("NUMBER OF TOKENS IN POSITIVE CLASS:", self.class_total_word_counts["pos"])
        print ("NUMBER OF TOKENS IN NEGATIVE CLASS:", self.class_total_word_counts["neg"])
        print ("VOCABULARY SIZE: NUMBER OF UNIQUE WORDTYPES IN TRAINING CORPUS:", len(self.vocab))

    def update_model(self, bow, label):
        """
        IMPLEMENT ME! (Question 1)

        Update internal statistics given a document represented as a bag-of-words
        bow - a Counter mapping from words to their counts
        label - the class of the document whose bag-of-words representation was input
        
        This function doesn't return anything but should update a number of internal
        statistics. Specifically, it updates:
          - the type-level word counts for each label (self.class_word_counts)
          - the number of word tokens seen for each label (self.class_total_word_counts)
          - the vocabulary seen so far (self.vocab)
          - the number of documents seen for each label (self.class_total_doc_counts)
        """
        for b in bow:
            if b not in self.vocab:
                self.vocab.add(b)
            if b in self.class_word_counts[label]:
                self.class_word_counts[label][b] += bow[b]
            else:
                self.class_word_counts[label][b] = bow[b]
            self.class_total_word_counts[label] += bow[b]
        self.class_total_doc_counts[label] += 1
        

    def tokenize_and_update_model(self, doc, label):
        """
        IMPLEMENT ME! (Question 1)

        Tokenizes a document doc and updates internal count statistics.
        doc - a string representing a document.
        label - the sentiment of the document (either postive or negative)
        
        Make sure to use self.tokenizer and self.update_model
        """
        bow = self.tokenizer(doc.lower())
        self.update_model(bow, label)

        
    def top_n(self, label, n):
        """
        IMPLEMENT ME! (Question 2)
        
        Returns a list of tuples corresponding to the most frequent n word types and
        their counts for documents with class 'label' in descending order.
        """
        sortedByCount = sorted(self.class_word_counts[label].items(),key=lambda w:w[1], reverse=True)
        takeFirstn = sortedByCount[:n]
        return takeFirstn
    

    def p_word_given_label(self, word, label):
        """
        IMPLEMENT ME! (Question 3)

        Returns the probability of the input word given the input label
        according to this NB model.
        """
        #print(self.class_total_word_counts)
        return  self.class_word_counts[label][word] / self.class_total_word_counts[label]
        

    def p_word_given_label_and_alpha(self, word, label, alpha):
        """
        IMPLEMENT ME! (Question 5)

        Returns the probability of word given label wrt psuedo counts.
        alpha - float; pseudocount parameter
        """
        return (self.class_word_counts[label][word] + alpha) / (self.class_total_word_counts[label] + alpha *len(self.vocab))
        

    def log_likelihood(self, bow, label, alpha):
        """
        IMPLEMENT ME! (Question 6)

        Returns the log likelihood of a tokenized document given a label
        and pseudocount.
        bow - a bag of words (i.e., a tokenized document)
        label - either the positive or negative label
        alpha - float; pseudocount parameter
        
        Make sure to use self.p_word_given_label_and_alpha
        """
        returns = float(0)
        for b in bow:
            returns += math.log(self.p_word_given_label_and_alpha(b, label, alpha)) * bow[b]
        return returns
    
    
    def log_prior(self, label):
        """
        DO NOT MODIFY THIS FUNCTION

        Returns the log prior of a document having the class 'label'.
        """
        label_docs = self.class_total_doc_counts[label]
        total_docs = sum(self.class_total_doc_counts.values())
        return math.log(label_docs / total_docs)
 

    def likelihood_ratio(self, word, alpha):
        """
        IMPLEMENT ME! (Question 9)

        Returns the ratio of P(word|pos) to P(word|neg).
        """
        return self.p_word_given_label_and_alpha(word, "pos", alpha)/self.p_word_given_label_and_alpha(word, "neg", alpha)


    def unnormalized_log_posterior(self, bow, label, alpha):
        """
        IMPLEMENT ME! (Question 12)

        Computes the unnormalized log posterior (of doc being of class 'label').
        bow - a bag of words (i.e., a tokenized document)
        """
        return self.log_prior(label) + self.log_likelihood(bow, label, alpha)
        

    def classify(self, bow, alpha):
        """
        IMPLEMENT ME! (Question 12)

        Compares the unnormalized log posterior for doc for both the positive
        and negative classes and returns the either POS_LABEL or NEG_LABEL
        (depending on which resulted in the higher unnormalized log posterior)
        bow - a bag of words (i.e., a tokenized document)
        """
        if self.unnormalized_log_posterior(bow, "pos", alpha) > self.unnormalized_log_posterior(bow, "neg", alpha):
            return "pos"
        else:
            return "neg"


    def evaluate_classifier_accuracy(self, alpha):
        """
        DO NOT MODIFY THIS FUNCTION

        alpha - pseudocount parameter.
        This function should go through the test data, classify each instance and
        compute the accuracy of the classifier (the fraction of classifications
        the classifier gets right.
        """
        correct = 0
        total = 0

        with open(self.test_fn, encoding="utf-8") as reader:
            for line in reader:
                _, label, text = line.strip().split('\t')
                bow = self.tokenizer(text)
                if self.classify(bow, alpha) == label:
                    correct += 1
                total += 1
        return 100 * correct / total

**Question 1 (5 pts)**

To start, implement the `update_model` and `tokenize_and_update_model` functions. Make sure to read the function comments so you know what to update. Also review the NaiveBayes class variables in the `def __init__` method of the NaiveBayes class  to get a sense of which statistics are important to keep track of. Once you have implemented `update_model` and `tokenize_and_update_model`, run the train model function using the code below. You’ll need to provide the path to the dataset you downloaded to run the code.

In [4]:
nb = NaiveBayes(train_tsv="movie_reviews_train.tsv", test_tsv="movie_reviews_test.tsv", tokenizer=simple_tokenizer)
nb.train_model()

if len(nb.vocab) == 76217:
    print("Great! The vocabulary size is {}".format(76217))
else:
    print("Oh no! Something seems off. Double check your code before continuing. Maybe a mistake in update_model?")

REPORTING CORPUS STATISTICS
NUMBER OF DOCUMENTS IN POSITIVE CLASS: 2000.0
NUMBER OF DOCUMENTS IN NEGATIVE CLASS: 2000.0
NUMBER OF TOKENS IN POSITIVE CLASS: 470375.0
NUMBER OF TOKENS IN NEGATIVE CLASS: 458554.0
VOCABULARY SIZE: NUMBER OF UNIQUE WORDTYPES IN TRAINING CORPUS: 76217
Great! The vocabulary size is 76217


### Exploratory analysis
Let's begin to explore the count statistics stored by the update model function. 

**Question 2 (5 pts)**

**(a)** Implement `top_n` function for the NaiveBayes class to find the top 10 most common words in the positive class and top 10 most common words in the negative class.

In [5]:
for label in ["pos", "neg"]:
    print("TOP 10 WORDS FOR CLASS '{}':".format(label))
    for word, count in nb.top_n(label, 10):
        print("  {}:\t{}".format(word, count))
    print()

TOP 10 WORDS FOR CLASS 'pos':
  the:	27401
  and:	14265
  a:	13056
  of:	12255
  to:	10521
  is:	8872
  in:	7904
  i:	5661
  that:	5384
  this:	5356

TOP 10 WORDS FOR CLASS 'neg':
  the:	25700
  a:	12634
  and:	11498
  of:	10883
  to:	10819
  is:	7778
  in:	6822
  i:	6480
  this:	5980
  that:	5439



**(b)** What is the first thing that you notice when you look at the top 10 words for the 2 classes? Are these words helpful for discriminating between the two classes? Do you imagine that processing other English text will result in a similar phenomenon? What about other languages?

**Your answer here**  
All the top 10 words in both classes are same and they are natural words which we are speaking with. However, these words do not have actural meaning for us to discrimination between positive and negative classes. I do think that processing other English text will result in a similar phenomenon. Also, for Chinese, it would be same. 

**Question 3 (5 pts)**

The Naive Bayes model assumes that all features are conditionally independent given the class label. For our purposes, this means that the probability of seeing a particular word in a document with class label $y$ is independent of the rest of the words in that document.

 **(a)** Implement the `p_word_given_label` function. This function calculates $\Pr(w|y)$ (i.e., the probability of seeing word w in a document given the label of that document is y).

Use your `p_word_given_label` function to compute the probability of seeing the word *loved* given each sentiment label. Repeat the computation for the word *terrible.* 

In [6]:
for term in ["loved", "terrible"]:
    for label in ["pos", "neg"]:
        print("Pr({}|{}):\t{:.3e}".format(term, label, nb.p_word_given_label(term, label)))

Pr(loved|pos):	3.146e-04
Pr(loved|neg):	1.330e-04
Pr(terrible|pos):	3.827e-05
Pr(terrible|neg):	3.009e-04


**(b)** Which word has a higher probability given the positive class, *loved* or *terrible*? Which word has a higher probability given the negative class? Is this what you would expect?

**Your answer here**  
Love would have a higher probability given the positive class and terrible would have a higher probability given the negative class.And I think that is the output that I expect.

**Question 4 (5 pts)**

In the next cell, compute the probability of the word *tender* in the positive training data and negative training data.

In [7]:
term = 'tender'
for label in ['pos', 'neg']:
    print("Pr({}|{}):\t{:.3e}".format(term, label, nb.p_word_given_label(term, label)))

Pr(tender|pos):	1.913e-05
Pr(tender|neg):	0.000e+00


What is unusual about $P(tender|neg)$? What would happen if we took the log of $P(tender|neg)$? What would happen if we multiplied $P(tender|neg)$ by $P(terrible|neg)$? Why might these operations cause problems for a Naive Bayes classifier?

**Your answer here**  
P(tender|neg) is 0 and 'tender' never show up in a negative class. Since P(tender|neg) = 0, log(P(tender|neg)) would be very close to -infinite. So if we calculate P('tender'|neg) * P('other word'|neg), it would also be 0 since the first term is 0.

**Question 5 (5 pts)**

We can address the issues from Question 4 by using add-$\alpha$ smoothing. Implement `p_word_given_label_and_alpha` for the NaiveBayes class and then run the next cell.

In [8]:
term = "tender"
for label in ['pos', 'neg']:
    print("Pr({}|{}):\t{:.3e}".format(term, label, nb.p_word_given_label_and_alpha(term, label, 0.1)))

Pr(tender|pos):	1.904e-05
Pr(tender|neg):	2.145e-07


### Priors, Likelihoods, and Posteriors

As noted before, the Naive Bayes model assumes that all words ($w_{d_i}$) in a document ($d$) are independent of one another given the document's label ($y_d$). Because of this we can write the likelihood of a document as:

$$\Pr(d) = \Pr(w_{d_1},\dots,w_{d_n}|y_d) = \prod_{i=1}^{n}\Pr(w_{d_i}|y_d)$$

However, if a document has a lot of words, the likelihood will become extremely small and we’ll encounter numerical underflow. So, we'll need want to work in log-space.

$$\ln \Pr(w_{d_1},\cdots,w_{d_n}|y_d) = \sum_{i=1}^{n}\ln \Pr(w_{d_i}|y_d)$$

**Question 6 (5 pts)**
Implement the `log_likelihood` using the above equation. 

There is nothing to print out for this question. But you will use these functions in a moment...

Naive Bayes is a model that tells us how to compute the posterior
probability of a document having some label (i.e.,
$\Pr(y_d|d)$). We can do this using Bayes' rule:

$$\Pr(y_d|d) = \frac{\Pr(y_d)\cdot\Pr(d|y_d)}{\Pr(d)}$$

Since we have functions to compute the log likelihood ($\ln\Pr(d|y_d)$) and log prior ($\ln\Pr(y_d)$), all we're missing is the *normalizer* $\Pr(d)$.

**Question 7 (5 pts)**
Derive the normalizer by expanding $\Pr(d)$. You will have to use "MathJax" to write out the equations. MathJax is very similar to LaTeX. 99% of the MathJax you will need to write for this course (and others at UMass) is included in the first answer of [this](https://math.meta.stackexchange.com/questions/5020/mathjax-basic-tutorial-and-quick-reference) tutorial. MathJax and LaTeX can be annoying first, but once you get a little practice, using these tools will feel like second nature.

**Your answer here**  
$P({d}) = \sum_{d=0}^n P({d}|y_d)P(y_d)$

**Question 8 (5 pts)**
One way to classify a document is to compute the unnormalized log posterior for both labels and take the argmax (i.e., the label that yields the higher unnormalized log posterior). The unnormalized log posterior is the sum of the log prior and the log likelihood of the document. Why don’t we need to compute the log normalizer in this case?


**Your answer here**  
Since the value of both class are same, it's not changing the value of argmax.

As we saw earlier, the 10 most frequent words for each class do not seem to tell us much about the classes. A much more informative metric, is the likelihood ratio, which is defined as

$$LR(w)=\frac{\Pr(w|pos)}{\Pr(w|neg)}$$

A word with $LR(w)=3$ is 3 times more likely to appear in the positive class than in the negative. A word with $LR(w)=0.33$ is one-third as likely to appear in the positive class as opposed to the negative class.

### Liklihood Ratios

**Question 9 (5 pts)** Implement the `likelihood_ratio` function and use it to investigate the likelihood ratio of the words *loved*, *terrible*, *the*, and *to*.

In [9]:
for term in ['loved', 'terrible', 'the', 'to']:
    print("LIKELIHOOD RATIO OF '{}':\t{:.2f}".format(term, nb.likelihood_ratio(term, 0.2)))

LIKELIHOOD RATIO OF 'loved':	2.36
LIKELIHOOD RATIO OF 'terrible':	0.13
LIKELIHOOD RATIO OF 'the':	1.04
LIKELIHOOD RATIO OF 'to':	0.95


**Question 10 (5 pts)** What is the minimum and maximum possible values the likelihood ratio can take? Does it make sense that $LR(loved) > LR(the)$ ?

**Your answer here**   
The minimum possible value of likelihood ratio is 0 and the maximum possible is unbounded.  
It makes sence that $LR('loved') > LR('the')$. Since 'love' looks like a positive word while 'the' is more like a natural word which would appear in both negitive and positive documents as almost same frequency.

**Question 11 (5 pts)** Find the word in the vocabulary with the highest likelihood ratio below. Your code must print the word and it's likelihood ratio following the format used in the Question 9.

In [10]:
# Your code here!
return_word = ''
max_like = 0.0
for word in nb.vocab:
    cur = nb.likelihood_ratio(word,0.2) 
    if cur > max_like:
        max_like = cur
        return_word = word
print("The highest likelihood ratio words is: "+return_word + ". The ratio is: "+ str(max_like) )

The highest likelihood ratio words is: emma. The ratio is: 113.17630962912442


### Classification

**Question 12 (5 pts)** Implement the `unnormalized_log_posterior` and `classify` functions. The `classify` function should use the unnormalized log posteriors but should not compute the normalizer.

Once you implement the `classify` function, we'd like to evaluate its accuracy. `evaluate_classifier_accuracy` is implemented for you so you don't need to change that method.

In [11]:
print(nb.evaluate_classifier_accuracy(0.2))

80.5


**Question 13 (5 pts)**

Try evaluating your model again with a smoothing parameter of 1000.

In [12]:
print(nb.evaluate_classifier_accuracy(1000))

68.0


Does the accuracy go up or down when $\alpha$ is raised to 1000? Why do you think this is?

**Your answer here**  
The accuracy go down when alpha raised to 1000.  
This would happen since it has been smoothed too much. Let's imagine a word appeared 100 times in 10000 counts with 500 size of vocab. The probility was 100/10000 = 1/100 before smoothing, but (100+1000)/(10000+1000*500) = 0.002156 which varies a lot and would effect the accuracy. 

**Question 14 (5 pts)** 

Find a review that your classifier got wrong.

**(a)** In the following cell, print out a review that your classifier got wrong. Print out the text of the review along with its label. 

In [37]:
# Your code here
with open(nb.test_fn, encoding="utf-8") as reader:
            for line in reader:
                _, label, text = line.strip().split('\t')
                bow = nb.tokenizer(text)
                tag = nb.classify(bow, 0.2)
                if tag != label:
                    print(text)

A handful of nubile young college sorority sisters decide to go camping with a professor. A giant druid want to sacrifice them to prevent the apocalypse come the year 2000, they also have to contend with bikers, an Indian and a loch ness monster type thing. Worth watching for only 3 reasons, George 'Buck' Flower (a sadly unsung B-movie staple) is on hand as a hobo and the other 2 belong to the stunning Savannah (in one of only 3 non-porn roles she had). Both have very small roles. Too bad everything else in the movie is horrendously bad. My Grade: D- Retromedia DVD Extras: Original Trailer Eye Candy: 4 pairs of breasts, 2 asses
"Night of the Hunted" stars French porn star Brigitte Lahaie.In fact,many of the cast members in this slow-moving production were porn actors at the time of its frantic filming.This film is certainly different than Rollin's usual lesbian vampire flicks,but it's not as memorable as for example "Lips of Blood" or "Fascination".Lahaie plays an amnesiac hitchhiker w

Yes, this bizarre feature was written by John Sayles. Shot in Toronto, it's yet another '80s era feature about the dangers of the urban jungle, where the police fear to go and the homeless and the criminal classes are the only inhabitants. Into this mix comes the myth of Wild Thing, a feral young man raised by a bag lady after his parents were murdered by a dirty cop on the take (Maury Chaykin) and Chopper, the local crime lord (Robert Davi). Stir in the local do-gooders (priest Sean Hewitt and clueless social worker Kathleen Quinlan), and you have a recipe for some rather unexciting action sequences. Davi is the standout amongst the cast, and cinematographer Rene Verzier does a pretty good job. Otherwise this is a rather lumpen action pic that won't satisfy action fans and will leaves Sayles' admirers slack-jawed.
Poor Jane Austen ought to be glad she's not around to see this dreadful wreck of an adaptation. So many great Jane Austen movies have come out recently that this one deserve

**(b)** What are two reasons the classifier might have misclassified this example? What improvements could you make that may help the naive Bayes classifier classify this example correctly? 

**Your answer here**  
First reason might be that our training set is too small for this classifier. 
The very first reason I could think of is the train data size is too small. for example words 'drink' 'gamble' 'heartbreaking'are negative words at most of the time which could bring the negative rate higher. I think larger training set is more accurate it's going to get

**Question 15 (5 pts)**
Often times we care about multi-class classification rather than binary classification. What changes would we need to make to the NaiveBayes `__init__` function to supprt 4-class classification? Explain why.

**Your answer here**  
For class_total_doc_counts since we have 4 class here is 4 numbers  
For class_total_word_counts since we have 4 class here is 4 numbers  
For self.class_word_counts since we have 4 class here going to be 4 dict  
4+4+4*len(vocab)   

**Extra credit (up to 10 points)**

If you don't want to complete the extra credit, you can stop here! Otherwise, keep reading...

In this homework, we used whitespace tokenization to create a bag-of-words representation for our the movie reviews. It is possible to improve this represetation to improve your classifier's performance. Implement `tokenize_doc_and_more` using your own code and/or an external library such as `nltk` to perform tokenization, text normalization, word filtering, etc. Then, show improvement by running the cells below.

Roughly speaking, the larger performance improvement, the more extra credit. We will also give points for the quality of the evaluation and analysis process. We've provided a validation set (`movie_reviews_dev.tsv`) that you may use to test different representations. You can also provide some qualitative examples you found in the dataset to support your choices on preprocessing steps. Whatever you choose to try, make sure to describe your method and the reasons that you hypothesize for why the method works. You can use this ipython notebook to show your work. Be sure to explain what your code is doing.

In [28]:
# Feel free to import any external libraries in this code cell
from nltk.tokenize import RegexpTokenizer 
from nltk.stem.snowball import SnowballStemmer 
from nltk.util import ngrams
stemmer = SnowballStemmer('english')

def tokenize_doc_and_more(text): 
    """
      Return some representation of this text.
      At a minimum, you need to perform tokenization, the rest is up to you. 
    """
  # Implement me!
  # your code goes here
    bow = defaultdict(float)
    tokenizer = RegexpTokenizer('\w+|\$[\d\.]+|\S+') # make a tokenizer copied from RegexpTokenizer document
    bow = list(ngrams(tokenizer.tokenize(doc), 2)) # create bi-grams I have tried 2 3 5 and 10 the 2 gives the highest accurancy
    bow = Counter(bow)
    return bow

In [39]:
nb = NaiveBayes(train_tsv="movie_reviews_train.tsv", test_tsv="movie_reviews_test.tsv",
                tokenizer=simple_tokenizer)
nb.train_model()
nb.evaluate_classifier_accuracy(1)

REPORTING CORPUS STATISTICS
NUMBER OF DOCUMENTS IN POSITIVE CLASS: 2000.0
NUMBER OF DOCUMENTS IN NEGATIVE CLASS: 2000.0
NUMBER OF TOKENS IN POSITIVE CLASS: 470375.0
NUMBER OF TOKENS IN NEGATIVE CLASS: 458554.0
VOCABULARY SIZE: NUMBER OF UNIQUE WORDTYPES IN TRAINING CORPUS: 76217


81.4

Use cells at the bottom of this notebook to explain what you did in `tokenize_doc_and_more`. Include any experiments or explanations that you used to decide what goes in your function. Doing a good job examining, explaining, and justifying your work with small experiments and comments is as important as making the accuracy number go up!

In [40]:
# Your experiments and explanations go here